In [2]:
import cv2
import numpy as np

In [3]:
img_np = np.ones([5,5])

print(img_np)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [4]:
kernel = np.array([-1, 0, 1 ])
print(kernel)

[-1  0  1]


In [5]:
# 디지터 신호처리의 FIR 필터 한 번 살펴보면 좋음
# cv2는 OpenCV라는 C++로 작성된 라이브러리를 python 가져다 사용
# 그러므로 python의 행렬 정보를 OpenCV용으로 변환할 필요가 있음
# 목적 : 타입 컨버팅

img_cv = cv2.resize(img_np, (5,5))
                    
print(img_cv)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [6]:
kernel = cv2.resize(kernel, (1, 3))
print(kernel)

[[-1]
 [ 0]
 [ 1]]


In [9]:
img_cv[0][0] = 0
img_cv[2][2] = 2
img_cv[4][4] = 3

# filter2D - Convolution이 결국
# 라플라스 변환과 푸리에 변환에 관계를 가지고 있기 때문
# 푸리에 변환이 완료된 두 함수의 곱은 시간 영역에서 컨벌류션과 동일하다.
# 주파수 영역으로 변환한 두 함수의 곱이 시간 영역에서 컨벌루션과 동일하다.
# (증명: 자코비안 행렬과 라플라스 변환이 필요함)
conv_test = cv2.filter2D(img_cv, -1, kernel)

print(conv_test)

[[ 0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0. -1.  0.  2.]
 [ 0.  0.  0.  0.  0.]]


In [ ]:
import copy

def filter_2d(input_img, kernel):
    imput_img = copy.deepcopy(input_img)